In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Embedding,LSTM


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving IMDB Dataset.csv to IMDB Dataset.csv


In [ ]:
df = pd.read_csv("IMDB Dataset.csv")

In [ ]:
df["sentiment"].replace({"positive": 1, "negative": 0}, inplace=True)

x = np.array(df["review"].values)
y = np.array(df["sentiment"].values)

x_filtered = []

for review in x:

    #lowercasing the sentence
    review = review.lower()

    # removing punctuations from sentence
    for i in review:
        punc = '''  !()-[]{};:'"\,<>./?@#$%^&*_~  '''
        if i in punc :
            review = review.replace(i, " ")

    x_filtered.append(review)

In [ ]:
# One-Hot Encoding each sentence
vocabulary_size = 5000
onehot_encoded = [one_hot(review,vocabulary_size) for review in x_filtered]

# Padding each encoded sentence to have a max_length=500
max_length=500
x_padded = pad_sequences(onehot_encoded,max_length,padding="post")

x_train,x_test,y_train,y_test = train_test_split(x_padded,y,test_size=0.2)

In [ ]:
from keras.layers import Bidirectional

model = Sequential()
embedding_vector_size = 35
model.add(Embedding(vocabulary_size, embedding_vector_size, input_length=max_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 35)           175000    
                                                                 
 bidirectional (Bidirection  (None, 200)               108800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 284001 (1.08 MB)
Trainable params: 284001 (1.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
63/63 [==============================] - 60s 866ms/step - loss: 0.6939 - accuracy: 0.4942 - val_loss: 0.6934 - val_accuracy: 0.4820
Epoch 2/10
63/63 [==============================] - 48s 769ms/step - loss: 0.6652 - accuracy: 0.6418 - val_loss: 0.5886 - val_accuracy: 0.6840
Epoch 3/10
63/63 [==============================] - 45s 721ms/step - loss: 0.4687 - accuracy: 0.7794 - val_loss: 0.6155 - val_accuracy: 0.7160
Epoch 4/10
63/63 [==============================] - 44s 708ms/step - loss: 0.2983 - accuracy: 0.8879 - val_loss: 0.6185 - val_accuracy: 0.7540
Epoch 5/10
63/63 [==============================] - 45s 719ms/step - loss: 0.1875 - accuracy: 0.9360 - val_loss: 0.7080 - val_accuracy: 0.6380
Epoch 6/10
63/63 [==============================] - 49s 773ms/step - loss: 0.1025 - accuracy: 0.9655 - val_loss: 0.8337 - val_accuracy: 0.6720
Epoch 7/10
63/63 [==============================] - 46s 730ms/step - loss: 0.0546 - accuracy: 0.9870 - val_loss: 0.8760 - val_accuracy: 0.7180

In [ ]:
model.save("IMDB_sentiment_analysis_model")

In [ ]:

from keras.models import load_model

trained_model = load_model("IMDB_sentiment_analysis_model")
predicted = trained_model.predict(x_test)[2]

sentiment = 1 if predicted > 0.6 else 0

print("PREDICTED : ",sentiment)
print("ACTUAL : ",y_test[2])

16/16 [==============================] - 4s 187ms/step
PREDICTED :  1
ACTUAL :  1


In [ ]:
def get_sentiment(sentence: str):
    if isinstance(sentence, (str)):
        pass
    else:
        raise Exception("Input needs to be of type 'str' ")

    # filtering the sentence
    sentence = sentence.lower()

    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

    for word in sentence:
        if word in punc:
            sentence = sentence.replace(word, " ")

    # Loading the saved trained model.
    from keras.models import load_model

    trained_model = load_model("IMDB_sentiment_analysis_model")

    predicted = trained_model.predict(x_test)[2]
    sentiment = 1 if predicted > 0.6 else 0

    if sentiment == 1:
        print("Positive")
    else:
        print("Negative")

    return sentiment


get_sentiment("That movie was really good!")


16/16 [==============================] - 4s 211ms/step
Positive


1